In [1]:
import pandas as pd

df_fake = pd.read_csv('data\\Fake.csv')
df_fake.head()
df_fake['label'] = 0  # Gán nhãn: 0 là fake

df_real = pd.read_csv('data\\True.csv')
df_real['label'] = 1  # Gán nhãn: 1 là real

# Gộp 2 dataframe
df = pd.concat([df_fake, df_real]).sample(frac=1).reset_index(drop=True)
df = df.dropna(subset=['title', 'text'])
df.head(100)


,title,text,subject,date,label
0,Trump Supporter ADMITS To Being Racist After ...,We all know that the only reason Donald Trump ...,News,"January 19, 2016",0
1,FL Pastor ‘Accidentally’ Shoots Wife In The H...,"According to Pastor Darrell Reid, he was clean...",News,"June 11, 2016",0
2,U.S. lawmakers push for action on human rights...,WASHINGTON (Reuters) - Seven U.S. senators urg...,politicsNews,"June 30, 2016",1
3,Trump touts Charter hiring that was in works f...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"March 24, 2017",1
4,Trump names additional senior White House aide...,WASHINGTON (Reuters) - U.S. Republican Preside...,politicsNews,"January 4, 2017",1
...,...,...,...,...,...
95,"U.S. Commerce Secretary says market access, pr...",HONG KONG (Reuters) - U.S. Commerce Secretary ...,worldnews,"September 27, 2017",1
96,Pope ends sensitive trip to Asia after seeking...,DHAKA (Reuters) - Pope Francis ended a diploma...,worldnews,"December 2, 2017",1
97,Trump’s Son Tries His Hand At Politics: Water...,"Donald Trump s son, Eric Trump, wants to be ju...",News,"February 9, 2016",0
98,Irish government set to fall weeks before Brex...,DUBLIN (Reuters) - Ireland s minority governme...,worldnews,"November 24, 2017",1


In [4]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess(text):
    text = re.sub(r'\W', ' ', text.lower())
    words = text.split()
    return ' '.join([lemmatizer.lemmatize(w) for w in words if w not in stop_words])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LAPTOP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LAPTOP\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\LAPTOP\AppData\Roaming\nltk_data...


In [5]:
df['text'] = (df['title'] + " " + df['text']).apply(preprocess)

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['text'])  # Ma trận đặc trưng
y = df['label']  # Nhãn

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Tách train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Huấn luyện
model = LogisticRegression()
model.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4735
           1       0.99      0.99      0.99      4245

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980

